# SRTT Chunking Analysis (Wymbs/Mucha)

This notebook demonstrates trial-wise chunk detection for one SRT participant file using:
- chain networks over IKIs within each block
- multilayer community detection across blocks
- chunk metrics (`q_single_trial`, `phi`, `phi_normalized`) and null-model validation

The implementation is in `src/chunking.py`.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.chunking import (
    build_trial_network,
    extract_ikis,
    load_srt_file,
    run_full_analysis,
)

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True

In [ ]:
# Pick one participant file from SRT folder
srt_dir = Path("SRT")
all_files = sorted(srt_dir.glob("*.csv"))
if not all_files:
    raise FileNotFoundError("No SRT CSV files found in SRT/")

example_file = all_files[0]
print(f"Using file: {example_file}")

# Run full blue-sequence chunking analysis
results = run_full_analysis(
    example_file,
    sequence_type="blue",
    gamma=0.9,
    C=0.03,
    n_iter=100,
    n_permutations=100,
    random_state=42,
)

metrics = results["metrics"]
metrics.head()

In [ ]:
# Plot IKI trajectories across blocks
iki_matrix = np.vstack(metrics["ikis"].to_list())

fig, ax = plt.subplots(figsize=(11, 4))
for i in range(iki_matrix.shape[1]):
    ax.plot(metrics["block_number"], iki_matrix[:, i], alpha=0.6, label=f"IKI_{i+1}")
ax.set_title("IKI trajectories across blue-sequence blocks")
ax.set_xlabel("Block number")
ax.set_ylabel("IKI (seconds)")
ax.legend(ncol=4, fontsize=8)
plt.show()

In [ ]:
# Visualize one trial network as weighted adjacency matrix
trial_idx = 0
trial_ikis = np.array(metrics.loc[trial_idx, "ikis"])
g = build_trial_network(trial_ikis)

adj = np.array(g.get_adjacency(attribute="weight").data)
fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(adj, cmap="viridis")
ax.set_title(f"Trial network weights (block {metrics.loc[trial_idx, 'block_number']})")
ax.set_xlabel("IKI node")
ax.set_ylabel("IKI node")
fig.colorbar(im, ax=ax)
plt.show()

In [ ]:
# Plot chunk assignments over trials (rows=trials, cols=IKI positions)
labels = np.vstack(metrics["community_labels"].to_list())

fig, ax = plt.subplots(figsize=(10, 5))
im = ax.imshow(labels, aspect="auto", cmap="tab20")
ax.set_title("Chunk assignments across trials")
ax.set_xlabel("IKI position")
ax.set_ylabel("Trial index")
ax.set_xticks(np.arange(labels.shape[1]))
ax.set_xticklabels([f"IKI_{i+1}" for i in range(labels.shape[1])], rotation=45)
fig.colorbar(im, ax=ax, label="Community label")
plt.tight_layout()
plt.show()

In [ ]:
# Plot normalized chunk magnitude over training
fig, ax = plt.subplots(figsize=(11, 4))
ax.plot(metrics["block_number"], metrics["phi_normalized"], marker="o", ms=3, lw=1)
ax.axhline(0.0, color="black", linestyle="--", linewidth=1)
ax.set_title("Normalized chunk magnitude (phi_normalized) over blocks")
ax.set_xlabel("Block number")
ax.set_ylabel("phi_normalized")
plt.show()

In [ ]:
# Inspect summary stats and null-model validation
summary_cols = [
    "block_number",
    "q_single_trial",
    "phi",
    "phi_normalized",
    "n_chunks",
    "chunk_boundaries",
]
print(metrics[summary_cols].head(10))

print("\nValidation metrics:")
for key, value in results["validation"].items():
    if key == "null_scores":
        print(f"{key}: <{len(value)} scores>")
    else:
        print(f"{key}: {value}")